In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
MODEL_PATH = "my_model.h5"  
model = load_model(MODEL_PATH)

In [ ]:
def prepare_digit(image):
    """Preprocess ROI from webcam for digit recognition (MNIST style)."""
    # Convert to grayscale + blur to reduce noise
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray_img, (5, 5), 0)

    # Adaptive thresholding (invert colors: digit = white)
    binary = cv2.adaptiveThreshold(
        blurred, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 11, 3
    )

    # Reduce noise and emphasize strokes
    binary = cv2.medianBlur(binary, 3)
    binary = cv2.dilate(binary, np.ones((3, 3), np.uint8), iterations=1)

    # Find digit contour
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Biggest contour = digit
        main_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(main_contour)

        # Create square canvas around digit
        side = max(w, h)
        dx = (side - w) // 2
        dy = (side - h) // 2

        square_digit = np.zeros((side, side), dtype=np.uint8)
        square_digit[dy:dy + h, dx:dx + w] = binary[y:y + h, x:x + w]

        # Resize to 20×20 and place in 28×28 (padding = 4 px)
        resized = cv2.resize(square_digit, (20, 20))
        canvas = np.zeros((28, 28), dtype=np.uint8)
        canvas[4:24, 4:24] = resized
    else:
        # Fallback: just resize full ROI
        canvas = cv2.resize(binary, (28, 28))

    # Normalize and reshape for model input
    normalized = canvas.astype("float32") / 255.0
    normalized = normalized.reshape(1, 784)  # (1,784) for fully-connected MNIST

    return normalized, canvas


In [4]:
def run_webcam_prediction(model):
    cam = cv2.VideoCapture(0)
    if not cam.isOpened():
        print("❌ Webcam not available")
        return

    print("✅ Webcam active! Draw a digit inside the green box. Press 'q' to quit.")

    while True:
        grabbed, frame = cam.read()
        if not grabbed:
            break

        height, width = frame.shape[:2]
        box_size = 150
        x_start = (width - box_size) // 2
        y_start = (height - box_size) // 2
        x_end, y_end = x_start + box_size, y_start + box_size

        # Draw ROI guide box
        output_frame = frame.copy()
        cv2.rectangle(output_frame, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)

        # Process ROI
        roi = frame[y_start:y_end, x_start:x_end]
        processed_input, debug_view = prepare_digit(roi)
        preds = model.predict(processed_input, verbose=0)
        pred_digit = np.argmax(preds)
        pred_conf = np.max(preds)

        # --- New Output Style ---
        # Black banner at bottom
        cv2.rectangle(output_frame, (0, height - 60), (width, height), (0, 0, 0), -1)

        # Display prediction
        cv2.putText(output_frame, f"Digit: {pred_digit}", (20, height - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)   # Yellow
        cv2.putText(output_frame, f"Confidence: {pred_conf:.2f}", (200, height - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2) # White

        # Place CNN view in bottom-right
        preview = cv2.resize(debug_view, (120, 120))
        preview = cv2.cvtColor(preview, cv2.COLOR_GRAY2BGR)
        output_frame[height-130:height-10, width-130:width-10] = preview
        cv2.putText(output_frame, "Model Input", (width-150, height-140),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Show final window
        cv2.imshow("Digit Recognition (Webcam)", output_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cam.release()
    cv2.destroyAllWindows()


In [ ]:
import tensorflow as tf

if __name__ == "__main__":
    # Load your trained MNIST model
    model = tf.keras.models.load_model("my_model.h5")

    # Pass the model into the function
    run_webcam_prediction(model)


✅ Webcam active! Draw a digit inside the green box. Press 'q' to quit.
